# 🎬 Advanced Content-Based Movie Recommendation Engine (NLP)
**By: Vishwaraj Khatpe | Final Year Engineering Student (AIML)**

---

## 1. Executive Summary
This project addresses the "cold start" problem in recommendation systems—how to suggest movies to users without needing their historical viewing data. By leveraging Natural Language Processing (NLP) techniques on movie plot summaries, this engine identifies semantic similarities between films that simple genre-matching cannot capture.

---

## 2. Technical Approach
Instead of traditional metadata clustering, this system utilizes a Vector Space Model:

* **Text Ingestion:** Processing raw text from movie "overviews" (plot summaries).
* **TF-IDF Vectorization:** Converting unstructured text into a mathematical feature matrix, weighing unique plot points higher than common words.
* **Cosine Similarity:** Calculating the angular distance between movie vectors to quantify semantic relatedness.



---

## 3. Why This Approach?

* **Granularity:** While *Interstellar* and *Star Wars* are both "Sci-Fi", NLP can distinguish that one is about "black holes/physics" and the other is a "space opera."
* **Scalability:** Matrix-based filtering is highly optimized for large datasets.

### [Code] - Imports

In [1]:
# Core Libraries for Data Manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Natural Language Processing & Machine Learning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel # Faster than cosine_similarity for dense matrices

# Configuration
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

print(" Libraries Imported Successfully")

 Libraries Imported Successfully


## 4. Data Ingestion & Cleaning

We load the dataset and perform critical cleaning steps to prepare the text data for vectorization.

### Key Decisions:

* **Memory Management:** The dataset can be large, so we process only necessary columns.
* **Text Sanitization:** Null values in text columns cause vectorizers to crash. We must fill NaN values with empty strings.

### [Code] - Data Loading

In [2]:
# Load dataset
try:
    df = pd.read_csv("../data/movies_metadata.csv", low_memory=False)
    print(f"Original Dataset Shape: {df.shape}")
except FileNotFoundError:
    print("Error: 'movies_metadata.csv' not found. Please upload the file.")

# Select distinct features for the recommendation engine
# 'overview' is our primary feature for NLP
selected_cols = ['title', 'overview', 'genres', 'vote_average', 'popularity']
movies = df[selected_cols].copy()

# Data Cleaning
# 1. Drop movies with no title (critical identifier)
movies = movies.dropna(subset=['title'])

# 2. Fill Null Plot Summaries
# TF-IDF cannot handle NaN; we replace them with empty strings
movies['overview'] = movies['overview'].fillna('')

# 3. Clean numeric columns for display purposes later
movies['popularity'] = pd.to_numeric(movies['popularity'], errors='coerce')
movies['vote_average'] = pd.to_numeric(movies['vote_average'], errors='coerce')

# Reset index to ensure robust lookups later
movies = movies.reset_index(drop=True)

print(f"Cleaned Dataset Shape: {movies.shape}")
movies[['title', 'overview']].head(3)

Original Dataset Shape: (45466, 24)
Cleaned Dataset Shape: (45460, 5)


,title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...


# **NLP Feature Engineering**


## 5. Feature Engineering: TF-IDF Vectorization

To compare plot summaries mathematically, we must convert text to numbers. We use **TF-IDF (Term Frequency-Inverse Document Frequency)**.

### Technical Insight: Why TF-IDF and not simple Word Counts?

* **Term Frequency (TF):** Counts how often a word appears in a specific movie plot.
* **Inverse Document Frequency (IDF):** Penalizes words that appear in every movie (like "the", "movie", "plot"). This ensures that generic words don't dominate the similarity score, allowing unique keywords (e.g., "wizard", "dinosaur", "heist") to drive recommendations.

### [Code] - TF-IDF Matrix Construction

In [4]:
# Initialize TF-IDF Vectorizer
# stop_words='english': Removes common words like 'the', 'a', 'in'
tfidf = TfidfVectorizer(stop_words='english')

print("Constructing TF-IDF Matrix... (This may take a moment)")

# Fit and Transform the 'overview' column
# This creates a matrix where Rows = Movies, Columns = Unique Words
tfidf_matrix = tfidf.fit_transform(movies['overview'])

print(f"TF-IDF Matrix Created. Shape: {tfidf_matrix.shape}")
print(f"- Analysis: We are comparing {tfidf_matrix.shape[0]} movies across {tfidf_matrix.shape[1]} unique words.")

Constructing TF-IDF Matrix... (This may take a moment)
TF-IDF Matrix Created. Shape: (45460, 75827)
- Analysis: We are comparing 45460 movies across 75827 unique words.


## 6. Computing Similarity

With our text converted to vectors, we calculate the similarity between every pair of movies.

We use the **Linear Kernel** (equivalent to Cosine Similarity for normalized data). This computes the cosine of the angle between two vectors:

* **Score = 1.0**: The plots are identical.
* **Score = 0.0**: The plots share no meaningful words.



> **Note:** The resulting matrix is massive ($N \times N$ movies), containing the similarity score of every movie against every other movie.

### [Code] - Cosine Similarity Matrix

In [5]:
print(" Computing Cosine Similarity Matrix...")

# Compute the linear kernel (dot product) of the matrix
# Since TF-IDF output is normalized, this is equivalent to Cosine Similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

print(f" Similarity Matrix Computed. Shape: {cosine_sim.shape}")

# Create a reverse mapping of indices and movie titles
# This allows us to look up the index of a movie by its title
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()

 Computing Cosine Similarity Matrix...


MemoryError: Unable to allocate 15.4 GiB for an array with shape (45460, 45460) and data type float64

# **Inference Engine**


## 7. Building the Recommendation Function

This function encapsulates the core logic of the system through a four-step process:

1.  **Lookup**: Find the unique index of the user's input movie title in the metadata DataFrame.
2.  **Similarity Retrieval**: Access the specific row of similarity scores for that movie from the `cosine_sim` matrix.
3.  **Ranking**: Sort all movies based on their similarity scores in descending order to identify the closest matches.
4.  **Fetch**: Filter out the input movie itself and return the **top 10** most similar movie titles.

---


> **Implementation Note:** Ensure that the input title is pre-processed or matched using a case-insensitive search to prevent `IndexErrors` during the lookup phase.

### [Code] - Recommendation Function

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    """
    Returns top 10 recommended movies based on plot similarity.
    
    Args:
        title (str): Title of the movie (Case sensitive in this basic version).
    
    Returns:
        DataFrame: Similar movies with their plot overviews.
    """
    try:
        # 1. Get the index of the movie that matches the title
        idx = indices[title]
        
        # Handle duplicate titles (if any), taking the first one
        if isinstance(idx, pd.Series):
            idx = idx.iloc[0]

        # 2. Get the pairwsie similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))

        # 3. Sort the movies based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # 4. Get the scores of the 10 most similar movies (ignoring the first one which is itself)
        sim_scores = sim_scores[1:11]

        # 5. Get the movie indices
        movie_indices = [i[0] for i in sim_scores]

        # 6. Return the top 10 most similar movies
        return movies[['title', 'vote_average', 'overview']].iloc[movie_indices]
        
    except KeyError:
        return "⚠️ Movie not found. Please check the spelling exactly as it appears in the database."

# --- Test Case ---
test_movie = 'The Dark Knight Rises'
print(f"🎬 Recommendations for '{test_movie}':")
get_recommendations(test_movie)

# **Conclusion**

# 8. Conclusion & Future Improvements

## Project Summary
We successfully built a **Content-Based Filtering System** using Natural Language Processing (NLP). By processing plot summaries with **TF-IDF**, we enabled the system to "read" movie descriptions and understand semantic relationships, providing recommendations that go deeper than simple genre matching.

---

## Future Enhancements (Interview Talking Points)
To scale this project for a production environment, I would consider the following improvements:

### 1. Hybridization
Combining this **Content-Based** score with a **Collaborative Filtering (SVD)** model to incorporate user personalization and historical behavior.

### 2. Advanced Embeddings
Replacing TF-IDF with **BERT** or **Word2Vec** embeddings. This allows the model to capture deeper context (e.g., understanding the semantic relationship between terms like "king" and "queen").

### 3. Approximate Nearest Neighbors (ANN)
For datasets with millions of movies, calculating the full Cosine Similarity matrix is computationally expensive. Using libraries like **FAISS** or **Annoy** can reduce inference time:

* **Current complexity:** $O(N)$
* **Optimized complexity:** $O(\log N)$

> **Pro-Tip:** Mentioning ANN in interviews demonstrates that you understand the difference between a "laptop-scale" project and a "production-scale" system.